In [1]:
#@Time:2021/4/25 10:27
#@Author: 戴 妍 晴
#@File:3.py
#@Software:PyCharm
#coding:utf-8
###python3.5
import pandas as pd
global N
pcb = []

In [2]:
#输入进程数据
def inPcbFromInput():
    global N
    N = int(input("请输入进程数："))
    for i in range(N):
        print("****************************************")
        pName = int(input("请输入进程编号 ："))                 #进程编号
        inTime = int(input("请输入进入时间："))               #进入时间
        serverTime = int(input("请输入服务时间："))           #服务时间
        #数据按顺序存放到列表中 startTime=开始时间 finishTime=完成时间 zzTime=周转时间 dqzzTime=带权周转时间
        #          进程编号 进入时间    服务时间 开始  完成 就绪队列 周转 带权 等待  优先权/剩余运行时间
        pcb.append([pName, inTime, serverTime,    -1,     -1,    "",     0,  0,  0,       0])


In [3]:
#读取表格中的进程数据
def inPcbFromTable():
    global N
    Data = pd.read_excel(r'3.xlsx')  #读入表格数据
    print(Data)
    N = Data.shape[0]   #获取进程数
    print(N)
    for row in Data.itertuples():   #循环表格的每一行
#         print(row[1],row[2],row[3])
        #          进程编号     到达时间           服务时间 开始    完成 就绪队列 周转 带权 等待  优先权/剩余运行时间
        pcb.append([int(row[1]) ,int(row[2]) ,  int(row[3]) , -1,    -1 , "",      0,   0,   0     ,0])

In [4]:

# 时间片
def RR(t):
    pcb.sort(key=lambda x: x[1], reverse=False)  # 升序排序   x:x[1] 按照进入时间排序
    now = 0  # 现在时间
    done = 0  # 已完成进程数
    q = [0]  # 存储准备队列
    prel = []
    # 初始化 剩余时间
    for u in pcb:
        u[9] = u[2]  # 剩余时间==服务时间
    while done < 4:
        # 循环准备队列首个进程
        for i in q[:1]:
            # 若开始时间未设置
            if pcb[i][3] == -1:
                if i == 0:
                    pcb[0][3] = pcb[0][1]  # 开始时间 == 进入时间
                    now == pcb[0][3]
                else:
                    pcb[i][3] = now
            # 统计该进程的准备队列
            s = ""
            q.remove(i)
            if i + 1 in prel:
                prel.remove(i + 1)
            # 若剩余时间小于时间片
            if pcb[i][9] <= t:
                pcb[i][4] = now + pcb[i][9]  # 完成 =开始+服务
                now += pcb[i][9]
                pcb[i][9] = 0
                done += 1
                for x in range(i + 1, N):
                    # 判断进程是否到达
                    if (pcb[x][1] <= now) and (pcb[x][0] not in prel) and (pcb[x][9] != 0) and (x!=i):
                        prel.append(pcb[x][0])
                for j in prel:
                    s = s + " " + str(j)
                    pcb[i][5] = s
            else:
                pcb[i][9] -= t
                now += t
                for x in range(i + 1, N):
                    if (pcb[x][1] <= now) and (pcb[x][0] not in prel) and (pcb[x][9] != 0) and (x!=i):
                        prel.append(pcb[x][0])  # 2 3
                for j in prel:
                    s = s + " " + str(j)
                pcb[i][5] = s
                prel.append(pcb[i][0])
            print("time  Process Num.   Start Time   End Time   Ready Queue")
            print(f"{now: ^5}", f"{pcb[i][0]: ^15}", f"{pcb[i][3]: ^15}", f"{pcb[i][4]: ^10}", f"{pcb[i][5]: ^5}")
            q = [index-1 for index in prel]  #编号抓换至列表中排序需要从1-》0

    PrintR("RR")

In [5]:
#高响应比优先
def HRRN():
    global N
    pcb.sort(key=lambda x:x[1], reverse=False)  #升序排序   x:x[1] 按照进入时间排序
    for i in range(N):
        if(i==0):
            startTime = pcb[i][1] # 到达时间 = 开始时间
            pcb[i][3] = startTime  
            pcb[i][4] = startTime + int(pcb[i][2])   #结束 =开始+服务
            s = ""
            for x in range(i+1,N):
                s = s + " " +  str(pcb[x][0])
            pcb[i][5] = s

        else:
            #计算优先权
            for j in range(i,N):
                #若该进程已到达
                if pcb[j][1]<=pcb[i-1][4]:  
                    prior = (pcb[i-1][4]-pcb[j][1])/ pcb[j][2] +1  
                    pcb[j][9] = prior
            #按优先权从高到低排序
            
            pcb_part = sorted(pcb[i:],key=lambda x:x[9], reverse=True)  #降序排序   x:x[9] 按照优先权排序
            pcb[i:] = pcb_part   
            if pcb[i-1][4] >= pcb[i][1]:
                pcb[i][3] = pcb[i-1][4]  #开始 = 上一个的完成
            else:
                pcb[i][3] = pcb[i][1]
            

            pcb[i][4] = pcb[i][3] + pcb[i][2]   #完成 =开始+服务
            #统计准备队列
            s = ""
            for x in range(i+1,N):
                s = s + " " +  str(pcb[x][0])
            pcb[i][5] = s
    print("Process Num.    Start Time   End Time   Ready Queue")
    for i in range(N):
        print(f"{pcb[i][0]: ^15}",f"{int(pcb[i][3]): ^15}",f"{int(pcb[i][4]): ^10}",f"{pcb[i][5]: ^5}")
    PrintR("HRRN")
def PrintR(name):
    sumTAT = 0
    sumWTAT = 0
    sumWT = 0
    for i in range(N):  # 计算周转时间和带权周转时间
        pcb[i][6] = float(pcb[i][4]) - int(pcb[i][1])  # 周转时间
        pcb[i][7] = float(pcb[i][6]) / int(pcb[i][2])  # 带权周转时间
        if name == "RR":
            pcb[i][8] = pcb[i][4] - pcb[i][1]- pcb[i][2]
        else:
            pcb[i][8] = int(pcb[i][3] - pcb[i][1])  # 等待时间=开始-到达
        sumTAT = sumTAT + pcb[i][6]
        sumWTAT = sumWTAT + pcb[i][7]
        sumWT = sumWT + pcb[i][8]

    # 讲结果输出出来
    pcb.sort(key=lambda x: x[0], reverse=False)  ##升序排序  按照进程名排序
    print()

    print("Process Num.  Turnarround Time   WeightedTurnarround Time    Waiting Time")

    for i in range(N):
        print(f"{pcb[i][0]: ^15}", f"{int(pcb[i][6]): ^15}", f"{round(float(pcb[i][7]), 3): ^30}",
              f"{int(pcb[i][8]): ^15}")
    print()

    print("Algorithm    Average TAT   Average WTAT   Average WT")
    print(f"{name: ^15}", f"{float(sumTAT / N): ^15}", f"{round(float(sumWTAT / N), 3): ^10}",
          f"{float(sumWT / N): ^15}")


In [6]:
if __name__ == '__main__':
    x = int(input("请选择要进行的算法过程：\n 1-高响应比HRRN \n 2-时间片RR \n"))
    y = int(input("请选择输入进程数据方式：\n 1-键盘输入数据 \n 2-读入表格数据 \n"))
    if y==1:
        inPcbFromInput()  #键盘输入
    else:
        inPcbFromTable()   #读入表格
    if x == 1:
        HRRN() #高响应比
    elif x == 2:
        t = int(input("请输入时间片大小\n"))
        RR(t) #时间片
    else:
        print("输入错误")


请选择要进行的算法过程：
 1-高响应比HRRN 
 2-时间片RR 
2
请选择输入进程数据方式：
 1-键盘输入数据 
 2-读入表格数据 
2
   Process Num  Arrival Time     CPU Burst
0            1                0          3
1            2                1          6
2            3                2          1
3            4                3          5
4
请输入时间片大小
3
time  Process Num.   Start Time   End Time   Ready Queue
  3          1               0            3       2 3 4
time  Process Num.   Start Time   End Time   Ready Queue
  6          2               3            -1      3 4 
time  Process Num.   Start Time   End Time   Ready Queue
  7          3               6            7       4 2 
time  Process Num.   Start Time   End Time   Ready Queue
 10          4               7            -1       2  
time  Process Num.   Start Time   End Time   Ready Queue
 13          2               3            13       4  
time  Process Num.   Start Time   End Time   Ready Queue
 15          4               7            15       2  

Process Num.  Turnarrou